<a href="https://colab.research.google.com/github/rdkdaniel/Chinook-DB-Analysis/blob/main/Analysis_Chinook_i_e_Just_View_the_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ImportLibraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import sqlite3
import sqlalchemy 

In [2]:
%matplotlib inline

In [ ]:


#Import Data
#Data from Kaggle



In [3]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#Make a directory named “.kaggle”
! mkdir ~/.kaggle

In [7]:
#Copy the “kaggle.json” into this new directory
! cp kaggle.json ~/.kaggle/

In [8]:
#Allocate the required permission for this file
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
#Downloading dataset
! kaggle datasets download alaasedeeq/chinook

  0% 0.00/373k [00:00<?, ?B/s]
100% 373k/373k [00:00<00:00, 102MB/s]


**Unzip data**

In [10]:
!unzip chinook.zip

Archive:  chinook.zip
  inflating: chinook.db              


In [11]:
# creating file path
dbfile = 'chinook.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)

# Be sure to close the connection
con.close()

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [ ]:
#Better visualize the DB??

**Various Functions**

In [12]:
db = 'chinook.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn)

In [13]:
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c) 

In [14]:
def show_tables():
    q = '''
        SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)

show_tables()

,name,type
0,Album,table
1,Artist,table
2,Customer,table
3,Employee,table
4,Genre,table
5,Invoice,table
6,InvoiceLine,table
7,MediaType,table
8,Playlist,table
9,PlaylistTrack,table


**Selecting Albums to Purchase**

[Borrowed content]


The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four.

All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
Artist Name 	Genre


Regal 	Hip-Hop

Red Tone 	Punk

Meteor and the Girls 	Pop

Slim Jim Bites 	Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.


In [17]:
albums_to_purchase = '''
WITH 
    usa_tracks_sold AS
        (
         SELECT il.* FROM InvoiceLine il
         INNER JOIN invoice i ON il.invoice_id = i.invoice_id
         INNER JOIN customer c ON i.customer_id = c.customer_id
         WHERE c.country = "USA" 
        )
SELECT 
    g.name Genre, 
    COUNT(uts.InvoiceLine_id) "Number of Tracks Sold", 
    CAST(COUNT(uts.InvoiceLine_id) AS FLOAT)/(SELECT COUNT(*) FROM usa_tracks_sold) "Percentage of Tracks Sold"
FROM usa_tracks_sold uts
INNER JOIN track t ON uts.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 10;
'''

run_query(albums_to_purchase)

DatabaseError: ignored